In [4]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import LeakyReLU
import numpy as np
import matplotlib.pyplot as plt


In [5]:
# define the Generator model
def build_Generator(z_input: Input, label_input:Input):
  model = tf.keras.models.Sequential()
  # Dense layer
  model.add(tf.keras.layers.Dense(128,input_dim=latent_dim))
  model.add(LeakyReLU(alpha=0.2))
  model.add(tf.keras.layers.BatchNormalization(momentum= 0.8))
  # Dense layer
  model.add(tf.keras.layers.Dense(256))
  model.add(LeakyReLU(alpha=0.2))
  model.add(tf.keras.layers.BatchNormalization(momentum= 0.8))
  # Dense layer
  model.add(tf.keras.layers.Dense(512))
  model.add(LeakyReLU(alpha=0.2))
  model.add(tf.keras.layers.BatchNormalization(momentum= 0.8))

  model.add(tf.keras.layers.Dense(np.prod((28, 28, 1)), activation='tanh'))
  model.add(tf.keras.layers.Reshape((28,28,1)))

  model.summary()
  # the latent input vector z
  label_embedding =tf.keras.layers.Embedding(input_dim = 10, output_dim=latent_dim)(label_input)
  flat_embedding = tf.keras.layers.Flatten()(label_embedding)

  # combine the noise and label by element-wise multiplication
  model_input = tf.keras.layers.multiply([z_input, flat_embedding])
  image = model(model_input)

  return Model([z_input, label_input], image)


In [6]:
# define the Discriminator model
def build_Discriminator():
  image = Input(shape =(28,28,1))
  flat_image = tf.keras.layers.Flatten()(image)

  label_input = Input(shape=(1,), dtype='int32')

  label_embedding = tf.keras.layers.Embedding(input_dim=10, output_dim=28*28*1)(label_input)
  flat_embedding = tf.keras.layers.Flatten()(label_embedding)
  model_input = tf.keras.layers.multiply([flat_image, flat_embedding])


  model = tf.keras.models.Sequential(name ='discriminator')
  #model.add(tf.keras.layers.Flatten(input_shape = (28,28,1)))

  #Dense
  model.add(tf.keras.layers.Dense(256))
  model.add(tf.keras.layers.LeakyReLU(alpha =0.2))
  #Dense
  model.add(tf.keras.layers.Dense(128))
  model.add(tf.keras.layers.LeakyReLU(alpha =0.2))
  #Dense
  model.add(tf.keras.layers.Dense(units =1, activation='sigmoid'))
  model.summary()


  validity = model(model_input)
  return Model([image, label_input], validity)



In [7]:
#train model
def train(generator, discriminator,combined,steps, batch_size):
   #load data
   (x_train,x_labels),_ =mnist.load_data()
   # Rescale in [-1, 1] interval
   x_train = (x_train.astype(np.float32) - 127.5) / 127.5
   x_train = np.expand_dims(x_train, axis=-1)

   #discriminator label
   real = np.ones((batch_size, 1))
   fake = np.zeros((batch_size, 1))

   latent_dim = generator.input_shape[0][1]

   for step in range(steps):
        idx = np.random.randint(0, x_train.shape[0], batch_size)
        real_images, labels = x_train[idx], x_labels[idx]
        # Random batch of noise
        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        # Generate a batch of new images
        generated_images = generator.predict([noise, labels])

        # Train the discriminator
        discriminator_real_loss = discriminator.train_on_batch([real_images,labels], real)
        discriminator_fake_loss = discriminator.train_on_batch([generated_images,labels], fake)
        discriminator_loss = 0.5 * np.add(discriminator_real_loss,discriminator_fake_loss)

        # Train the generator
        # random latent vector z
        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        # Condition on labels
        sampled_labels = np.random.randint(0, 10, batch_size).reshape(-1, 1)

        generator_loss = combined.train_on_batch([noise,sampled_labels], real)

        print("%d [Discriminator loss: %.4f%%, acc.: %.2f%%] [Generatorloss: %.4f%%]" % (step,
                                                                                         discriminator_loss[0], 100 * discriminator_loss[1], generator_loss))


In [12]:
def plot_generated_image(generator):
  n=10
  digit_size =28
  #big array of images
  Image = np.zeros((digit_size * n, digit_size * n))

  latent_dim = generator.input_shape[0][1]
  # n*n random latent distributions
  noise = np.random.normal(0, 1, (n * n, latent_dim))
  sampled_labels = np.full(n * n, label, dtype=np.int64).reshape(-1, 1)
  generated_images = generator.predict([noise,sampled_labels])

  for i in range(n):
    for j in range(n):
       slice_i = slice(i * digit_size, (i + 1) * digit_size)
       slice_j = slice(j * digit_size, (j + 1) * digit_size)
       Image[slice_i, slice_j] = np.reshape(generated_images[i * n + j], (28, 28))



  #plot
  plt.figure(figsize=(6, 5))
  plt.axis('off')
  plt.imshow(Image, cmap='Greys_r')
  plt.show()
  plt.close()

In [14]:
# GAN
#with latent vector size =64, train size=50000
latent_dim = 64
optimizer = Adam(0.00002, 0.5)

# Build and compile the discriminator
discriminator = build_Discriminator()
print("Discriminator network")
print(discriminator.summary())
discriminator.compile(loss='binary_crossentropy',optimizer=optimizer,metrics=['accuracy'])
discriminator.trainable = False

# Generator
z = Input(shape=(latent_dim,))
label = Input(shape=(1,))
generator = build_Generator(z,label)
print("Generator network")
print(generator.summary())
generated_image = generator([z,label])

# The discriminator takes generated image as input and determines validity
real_or_fake = discriminator([generated_image,label])

combined = Model([z,label], real_or_fake)
combined.compile(loss='binary_crossentropy', optimizer=optimizer)
train(generator, discriminator, combined, steps=30,batch_size=100)







Model: "discriminator"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_21 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_15 (LeakyReLU)      │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_22 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_16 (LeakyReLU)      │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_23 (Dense)                │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

...

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_22      │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_7         │ (None, 1, 64)     │        640 │ input_layer_22[0… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_21      │ (None, 64)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_11          │ (None, 64)        │          0 │ embedding_7[0][0] │
│ (Flatten)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multiply_7          │ (None, 64)        │          0 │ input_layer_21[0… │
│ (Multiply)          │                   │            │ flatten_11[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sequential_3        │ (None, 28, 28, 1) │    578,704 │ multiply_7[0][0]  │
│ (Sequential)        │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 579,344 (2.21 MB)

 Trainable params: 577,552 (2.20 MB)

 Non-trainable params: 1,792 (7.00 KB)

None
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 93ms/step
0 [Discriminator loss: 0.6855%, acc.: 67.25%] [Generatorloss: 0.6871%]
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
1 [Discriminator loss: 0.6873%, acc.: 55.67%] [Generatorloss: 0.6873%]
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 
2 [Discriminator loss: 0.6877%, acc.: 49.30%] [Generatorloss: 0.6879%]
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 
3 [Discriminator loss: 0.6881%, acc.: 47.66%] [Generatorloss: 0.6880%]
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
4 [Discriminator loss: 0.6883%, acc.: 46.58%] [Generatorloss: 0.6880%]
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
5 [Discriminator loss: 0.6885%, acc.: 45.87%] [Generatorloss: 0.6880%]
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step 
6 [Discriminator loss: 0.6887%, acc.: 45.27%] [Generatorloss: 0.6880%]
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
7 [Discriminator loss: 0.6888%, acc.: 45.17%] [Generatorloss: 0.6881%]
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
8 [Discriminator loss: 0.6889%, acc.: 44.46%] [Generatorloss: 0.6880%]
4/4 ━━━━━━━━━━